In [8]:
import tarfile
import tempfile
from skimage import io
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from spatialOmics import SpatialOmics

import urllib.request
import tarfile

import pandas as pd
import pickle
import numpy as np
from matplotlib.colors import ListedColormap, Normalize

# create empty instance
so = SpatialOmics()
import random

import geojson
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import os
import cv2
from json import JSONDecodeError


from function_masks_PANNUKE import *

In [9]:
PATH_root = '/media/user/Watson/data_Lyon/Ovaires/'

In [10]:
PATH_mask = PATH_root+'4_Athena/3_hovernet_select/json/'
PATH_img = PATH_root+'4_Athena/1_patches_select/'

In [11]:
patients = os.listdir(PATH_img)
len(patients)

28

# so.spl

In [12]:
fmeta = PATH_root+'Ovarian_patients_status.csv'
if os.path.exists(fmeta) :
    df_fmeta =  pd.read_csv(fmeta, sep=',')
    if 'Unnamed: 0' in df_fmeta.columns :
        df_fmeta.drop(columns='Unnamed: 0',inplace=True)
else:
    df_fmeta = pd.DataFrame()
    for patient in patients : 
        if 'CTR' in patient :
            status = "CTR"
        else :
            status = "YO-PCD"
        new_row = pd.DataFrame({'Cohort':['Lyon'], 'ID_patient':[patient], 'status':[status]})
        df_fmeta = pd.concat([df, new_row])
    df_fmeta.to_csv(fmeta)


In [13]:
df_tot = pd.DataFrame()

for patient in patients :
    patches = os.listdir(PATH_img+patient)
    for patch in patches : 
        df_temp = pd.DataFrame({'ID_patient' :[patient], 'ID_file' : [patient+'/'+patch[:-4]]})
        df_tot = pd.concat([df_tot, df_temp])

In [14]:
fmeta = PATH_root+'Ovarian_patients_status.csv'
so.spl = pd.merge(df_tot, df_fmeta, on='ID_patient')
so.spl.set_index('ID_patient', inplace=True)

# so.img & so.mask

## so.img

In [15]:
so.images = dict()
for patient in patients :
    patches = os.listdir(PATH_img+patient)
    for patch in patches : 
        so.images[patient+'/'+patch[:-4]] = io.imread(PATH_img+patient+'/'+patch)
        so.images[patient+'/'+patch[:-4]] = so.images[patient+'/'+patch[:-4]].T


## so.masks

In [16]:
so.masks = dict()

for patient in patients :
    patches = os.listdir(PATH_img+patient)
    for patch in patches :
        PATH_tot = PATH_mask+patient+'/'+patch[:-4]+'.json'
        with open(PATH_tot) as f :
            nuclei = geojson.load(f)
        nuclei = nuclei['nuc']

        mask_background = function_mask_background(nuclei)
        mask_neoplasic = function_mask_neoplasic(nuclei)
        mask_inflammatory = function_mask_inflammatory(nuclei)
        mask_connective = function_mask_connective(nuclei)
        mask_dead = function_mask_dead(nuclei)   
        mask_non_neo_epi = function_mask_nne(nuclei) 
        cellmasks = function_mask_cellmasks(nuclei)

        cellmasks_binary = mask_neoplasic + mask_inflammatory + mask_connective + mask_dead + mask_non_neo_epi
        cellmasks_type = function_mask_cellmasks_type(nuclei)


        concat = np.stack((mask_background, mask_neoplasic, mask_inflammatory, mask_connective, mask_dead, mask_non_neo_epi, cellmasks, cellmasks_binary, cellmasks_type))

        so.masks[patient+'/'+patch[:-4]] = {"others":concat[0],"neoplastic" :concat[1], "inflammatory" : concat[2], "connective" : concat[3], "dead" : concat[4], "non-neoplastic_epithelial" : concat[5],"cellmasks" : concat[6],"cellmasks_binary" : concat[7],"cellmasks_type" : concat[8]}#[:-4]

# s.uns (unstructured data)

In [17]:
from matplotlib import cm 
so.uns = {'cmaps': {}, 'cmap_labels': {}}

# define default colormap
so.uns['cmaps'].update({'default': cm.Purples})

# cell_type_id colormap
cmap = ['black', 'red', 'green', 'blue', 'yellow', 'orange']
cmap_labels = {0: 'others', 1: 'neoplastic',  2: 'inflammatory', 3: 'connective', 4: 'dead', 5: 'non-neoplastic epithelial'}
cmap = ListedColormap(cmap)

so.uns['cmaps'].update({'cell_type_id': cmap})
so.uns['cmap_labels'].update({'cell_type_id': cmap_labels})

In [18]:
#if no background
# from matplotlib import cm 
# so.uns = {'cmaps': {}, 'cmap_labels': {}}

# # define default colormap
# so.uns['cmaps'].update({'default': cm.Purples})

# # cell_type_id colormap
# cmap = ['red', 'green', 'blue', 'yellow', 'orange']
# cmap_labels = {1: 'neoplastic',  2: 'inflammatory', 3: 'connective', 4: 'dead', 5: 'non-neoplastic epithelial'}
# cmap = ListedColormap(cmap)

# so.uns['cmaps'].update({'cell_type_id': cmap})
# so.uns['cmap_labels'].update({'cell_type_id': cmap_labels})

# s.obs

In [19]:
so.obs={}
for patient in patients :
    patches = os.listdir(PATH_img+patient)
    for patch in patches :
        PATH_tot = PATH_mask+patient+'/'+patch[:-4]+'.json'
        with open(PATH_tot) as f :
            nuclei = geojson.load(f)
        nuclei = nuclei['nuc']
        if len(nuclei) != 0 :
            so.obs[patient+'/'+patch[:-4]] = centroid(nuclei)
            so.obs[patient+'/'+patch[:-4]].index += 1 
            so.obs[patient+'/'+patch[:-4]]['cell_type'] = so.obs[patient+'/'+patch[:-4]]['cell_type'].astype('category')

# Save so object

In [15]:
import pickle
pickle.dump(so, open('so_50_med.pkl', 'wb'))